In [1]:
#output_file-to save the layout in file, show-display the layout , output_notebook-to configure the default output state  to generate the output in jupytor notebook.
from bokeh.io import output_file, show, output_notebook 
#ColumnDataSource makes selection of the column easier and Select is used to create drop down 
from bokeh.models import ColumnDataSource, Select
#Figure objects have many glyph methods that can be used to draw vectorized graphical glyphs. example of glyphs-circle, line, scattter etc. 
from bokeh.plotting import figure 
#To create intractive plot we need this to add callback method.
from bokeh.models import CustomJS 
#This is for creating layout
from bokeh.layouts import column, gridplot, row
from bokeh.models import Legend

output_notebook() #create default state to generate the output


Loading BokehJS ...

In [2]:
# Deepnote Bokeh
from bokeh.plotting import output_file, save
from IPython.display import IFrame
from IPython.core.display import display, HTML
import tempfile

def bokeh_deepnote_show(plot):
    tmp_output_filename = tempfile.NamedTemporaryFile(suffix='.html').name
    output_file(tmp_output_filename)
    save(plot)

    f = open(tmp_output_filename, "r")
    display(HTML(f.read()))

https://www.kaggle.com/shivanisinha/bokeh-part-1-creating-an-interactive-dropdown/data?search=gapminder&select=gapminder_tidy.csv

In [3]:
import numpy as np
import pandas as pd
import os

All dataframes need to be in the same folder (including the dataframe with All neighborhoods)

In [27]:
# files_folder = '../data/feelings/'
files_folder = './data/feelings/'

files = os.listdir(files_folder)

In [5]:
DFS = []

BCN_districts = ['Ciutat Vella',"L'Eixample",'Sants-Montjuïc','Les Corts','Sarrià-Sant Gervasi',
                 'Gràcia','Horta-Guinardó','Nou Barris','Sant Andreu','Sant Martí']

BCN_districts = ['All neighborhoods'] + BCN_districts

for i,fname in enumerate(files):
    dat_i = pd.read_csv(''.join([files_folder, fname]))
    dat_i = dat_i.rename(columns={'Unnamed: 0' : 'score', 
                                'f_tristesa' : 'sadness', 'f_estres' : 'stress', 'f_avorriment' : 'boredom',
                                'f_calma' : 'calm', 'f_energia' : 'energy', 'f_felicitat' : 'happiness'})
    dat_i['neighborhood'] = BCN_districts[i]
    DFS.append(dat_i)

dat1 = pd.concat(DFS)

In [6]:
neighborhood0 = BCN_districts[0]

gap1= pd.melt(dat1, id_vars = ['score', 'neighborhood'])
gap2 = gap1[gap1['neighborhood'] == neighborhood0]

test2 = gap2.pivot_table(index=['score'], columns=['variable']).reset_index()
test2.columns = test2.columns.get_level_values(1)
test2 = test2.rename(columns={'':'score'})

Overall = ColumnDataSource(data=dat1)
Curr = ColumnDataSource(data=test2)


#plot and the menu is linked with each other by this callback function
callback = CustomJS(args=dict(source=Overall, sc=Curr), code="""
var f = cb_obj.value
sc.data['calm']=[]
sc.data['stress']=[]
sc.data['happiness']=[]
sc.data['sadness']=[]
sc.data['energy']=[]
sc.data['boredom']=[]
sc.data['score']=[]
for(var i = 0; i <= source.get_length(); i++){
	if (source.data['neighborhood'][i] == f){
		sc.data['calm'].push(source.data['calm'][i])
        sc.data['stress'].push(source.data['stress'][i])
        sc.data['happiness'].push(source.data['happiness'][i])
        sc.data['sadness'].push(source.data['sadness'][i])
        sc.data['energy'].push(source.data['energy'][i])
        sc.data['boredom'].push(source.data['boredom'][i])
        sc.data['score'].push(source.data['score'][i])
	 }
}   
   
sc.change.emit();
""")

Neighborhoods = list(gap1['neighborhood'].unique())
menu = Select(options=Neighborhoods,value=neighborhood0, title = 'Neighborhood selection:')  # drop down menu

In [7]:
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap

TOOLS = ''

pairs = {0:['sadness','happiness'], 1:['stress','calm'], 2:['boredom', 'energy']}
x_range = gap1.score.unique().tolist()
y_range0 = (0,70)
plots = []

yaxis_visibility = [True, False, False]
yaxis_label = ['Ratio of answers (%)', '', '']

color_neg = '#4D376C'
color_pos = '#FFCA3A'

width_bar = 1
alpha_bar = 0.7

plot_width = 360
plot_height = 230

for feel in pairs.items():
    
    i = feel[0]
    if i != 0:
        y_range = p_feel.y_range
    else:
        y_range = y_range0
    
    neg_feel = feel[1][0]
    pos_feel = feel[1][1]
    
    # Initializing figure
    p_feel=figure(x_axis_label ='Frequency levels', y_axis_label = yaxis_label[i],
                  plot_width= plot_width, plot_height=plot_height,
                  x_range=x_range, y_range = y_range, tools = TOOLS) #creating figure object 

    # Creating bar glyphs
    r_neg = p_feel.vbar(x='score', top=neg_feel, width = width_bar, source= Curr, fill_alpha = alpha_bar,
                        fill_color=color_neg, muted_alpha=0.2, line_color = 'white')
    r_pos = p_feel.vbar(x='score', top=pos_feel, width = width_bar, source= Curr, fill_alpha = alpha_bar,
                        fill_color=color_pos, muted_alpha=0.2, line_color = 'white')

    # Creating legend    
    legend = Legend(items=[(neg_feel, [r_neg]),(pos_feel, [r_pos])], 
                    location="center", click_policy="hide",
                    border_line_color = None, background_fill_color = None, orientation = "horizontal")

    p_feel.add_layout(legend, 'above')
    
    # Grid
    p_feel.xgrid.visible = False
    p_feel.ygrid.visible = True
    p_feel.ygrid.grid_line_color = 'lightgrey'
    p_feel.ygrid.grid_line_dash = 'dotted'
    p_feel.ygrid.grid_line_alpha = 0.4
    
    # axis attributes
    p_feel.xaxis.major_tick_line_width = 0
        
    p_feel.yaxis.visible = yaxis_visibility[i]
    p_feel.yaxis.ticker = np.arange(y_range0[0],y_range0[1],10)
    p_feel.yaxis.major_tick_line_width = 0
    p_feel.yaxis.axis_line_width = 0
    
    p_feel.outline_line_color = None
    
    # Creating list to make row of plots
    plots.append(p_feel)
    
p = gridplot(plots, ncols = 3)
#p = row(plots)


In [8]:
menu.js_on_change('value', callback) # calling the function on change of selection

final_layout=gridplot([[None,menu,None],plots]) # creating the layout

show(final_layout) # displaying the layout

In [9]:
bokeh_deepnote_show(final_layout)

## Exporting plot

Need to export the __final_layout__ variable to html file...

In [10]:
# from bokeh.plotting import figure
# from bokeh.resources import CDN
# from bokeh.embed import file_html

# html = file_html(final_layout, CDN, "f-dynamic")

In [11]:
# html

'\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  \n  <head>\n    \n      <meta charset="utf-8">\n      <title>f-dynamic</title>\n      \n      \n        \n          \n        \n        \n          \n        <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-2.3.1.min.js" integrity="sha384-YF85VygJKMVnHE+lLv2AM93Vbstr0yo2TbIu5v8se5Rq3UQAUmcuh4aaJwNlpKwa" crossorigin="anonymous"></script>\n        <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-widgets-2.3.1.min.js" integrity="sha384-KKuas3gevv3PvrlkyCMzffFeaMq5we/a2QsP5AUoS3mJ0jmaCL7jirFJN3GoE/lM" crossorigin="anonymous"></script>\n        <script type="text/javascript">\n            Bokeh.set_log_level("info");\n        </script>\n        \n      \n      \n    \n  </head>\n  \n  \n  <body>\n    \n      \n        \n          \n          \n            \n              <div class="bk-root" id="6d69fd18-cd56-4143-af79-6eeba8600d52" data-root-id="1178"></div>\n            \n          \n  

In [13]:
# file_fdyn = open("file_fdyn.html","w")
# file_fdyn.write(html)
# file_fdyn.close()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5cb1a2c0-2f56-40d6-8008-fedeaf8b6a17' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>